# Deploy COA Objects in Transcend:   DB_Objects

In [ ]:
%addconnect name=Transcend, user=sh186014, host=tdprdcop3.td.teradata.com, props="LOGMECH=LDAP"

In [ ]:
%connect Transcend

In [ ]:
database adlste_coa;

# DB_Objects including CDS

In [ ]:
--Drop Table adlste_coa_stg.stg_dat_DB_Objects

In [ ]:
-- GLOBAL TEMP TABLE:
CREATE MULTISET GLOBAL TEMPORARY TABLE adlste_coa_stg.stg_dat_DB_Objects 
(  Site_ID          VARCHAR(128)  CHARACTER SET LATIN
 , WeekID           INTEGER
 , DatabaseName     VARCHAR(128)  CHARACTER SET UNICODE 
 , CurrPermGB_Rank  INTEGER       
 , CommentString    VARCHAR(1024) CHARACTER SET UNICODE 
 , MaxPermGB        DECIMAL(18,2)  
 , CurrentPermGB    DECIMAL(18,2)  
 , FilledPct        DECIMAL(9,4)
 , Table_Count      INTEGER    
 , View_Count       INTEGER   
 , Index_Count      INTEGER    
 , Macro_Count      INTEGER    
 , SPTrig_Count     INTEGER     
 , UDObject_Count   INTEGER       
 , Other_Count      INTEGER    
) NO PRIMARY INDEX 
  ON COMMIT PRESERVE ROWS;

In [ ]:
--DROP TABLE adlste_coa.coat_dat_DB_Objects

In [ ]:
-- PERM Table
CREATE MULTISET TABLE adlste_coa.coat_dat_DB_Objects 
(  Site_ID          VARCHAR(128)  CHARACTER SET LATIN
 , WeekID           INTEGER
 , DatabaseName     VARCHAR(128)  CHARACTER SET UNICODE 
 , CurrPermGB_Rank  INTEGER       COMPRESS(0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250)
 , CommentString    VARCHAR(1024) CHARACTER SET UNICODE 
 , MaxPermGB        DECIMAL(18,2) COMPRESS(0) 
 , CurrentPermGB    DECIMAL(18,2) COMPRESS(0) 
 , FilledPct        DECIMAL(9,4)  COMPRESS(0.0000)
 , Table_Count      INTEGER       COMPRESS(0) 
 , View_Count       INTEGER       COMPRESS(0)
 , Index_Count      INTEGER       COMPRESS(0) 
 , Macro_Count      INTEGER       COMPRESS(0) 
 , SPTrig_Count     INTEGER       COMPRESS(0)  
 , UDObject_Count   INTEGER       COMPRESS(0)    
 , Other_Count      INTEGER       COMPRESS(0) 
 , Run_ID           INTEGER
) PRIMARY INDEX (Site_ID, WeekID, DatabaseName);

In [ ]:
replace view adlste_coa.coa_dat_DB_Objects
as  locking row for access
select * from adlste_coa.coat_dat_DB_Objects

In [ ]:
REPLACE PROCEDURE adlste_coa.sp_dat_DB_Objects ( spversion VARCHAR(128) ) 
SQL SECURITY CREATOR
BEGIN 

    /* start setup audit process */
    
    DECLARE runid  INTEGER;
    DECLARE siteid VARCHAR(128);
    DECLARE tablename VARCHAR(128);
    DECLARE callingsp VARCHAR(128); 
    DECLARE startdate DATE;
    DECLARE enddate DATE;
    
    SET runid = 0;
    SET tablename = 'adlste_coa.coat_dat_DB_Objects';
    SET callingsp = 'adlste_coa.sp_dat_DB_Objects';
    SET siteid = 'Empty Table';
    SET startdate = DATE;
    SET enddate = DATE;

    SELECT 
     coalesce(Site_ID,'empty table')       
    ,min(substr(trim(WeekID),1,4)||'-'||substr(trim(WeekID),5,2)||'-'||
     cast(cast((cast(substr(trim(WeekID),7,1) as int)*7)-6 as int format'99') as char(2)) )(DATE) as StartDate
    ,max(substr(trim(WeekID),1,4)||'-'||substr(trim(WeekID),5,2)||'-'||
     cast(cast((cast(substr(trim(WeekID),7,1) as int)*7)-6 as int format'99') as char(2)) )(DATE) as EndDate
    INTO siteid, startdate, enddate
    FROM adlste_coa_stg.stg_dat_DB_Objects
    Group by 1
    ;
    
    /* end setup audit process */
    
    
    
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate,
                              'normal run', 'START',  '') ;

    DELETE FROM adlste_coa.coat_dat_DB_Objects 
    WHERE (Site_ID, WeekID) IN
    (Select Site_ID, WeekID From adlste_coa_stg.stg_dat_DB_Objects );
    
    Insert into adlste_coa.coat_dat_DB_Objects 
    Select 
      Site_ID          
    , WeekID          
    , DatabaseName           
    , CurrPermGB_Rank  
    , CommentString    
    , MaxPermGB        
    , CurrentPermGB    
    , FilledPct        
    , Table_Count      
    , View_Count       
    , Index_Count      
    , Macro_Count      
    , SPTrig_Count     
    , UDObject_Count   
    , Other_Count      
    , :runid
    from adlste_coa_stg.stg_dat_DB_Objects ; 
    
    delete from adlste_coa_stg.stg_dat_DB_Objects;
        
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate,
                              'normal run', 'END', '') ;

END;

In [ ]:
select top 10 * from adlste_coa.coat_dat_DB_Objects

In [15]:
select top 100 * from adlste_coa.coa_audit_log where TableName = 'adlste_coa.coat_dat_DB_Objects'
order by 1 desc

Run_ID,ParentRun_ID,Site_ID,TableName,StartDate,EndDate,CallingSP,SQLVersion,Run_Type,Run_Status,Run_Message,Run_User,Run_TS
787,786,APPLE50BLANEY,adlste_coa.coat_dat_DB_Objects,2020-05-22,2020-07-01,adlste_coa.sp_dat_DB_Objects,1.0,normal run,END,,SH186014,2020-07-06 16:29:40
786,0,APPLE50BLANEY,adlste_coa.coat_dat_DB_Objects,2020-05-22,2020-07-01,adlste_coa.sp_dat_DB_Objects,1.0,normal run,START,,SH186014,2020-07-06 16:29:40
785,784,APPLE46REDWOOD,adlste_coa.coat_dat_DB_Objects,2020-05-22,2020-07-01,adlste_coa.sp_dat_DB_Objects,1.0,normal run,END,,SH186014,2020-07-06 16:29:35
784,0,APPLE46REDWOOD,adlste_coa.coat_dat_DB_Objects,2020-05-22,2020-07-01,adlste_coa.sp_dat_DB_Objects,1.0,normal run,START,,SH186014,2020-07-06 16:29:34
783,782,APPLE48EDDIES,adlste_coa.coat_dat_DB_Objects,2020-06-15,2020-07-01,adlste_coa.sp_dat_DB_Objects,1.0,normal run,END,,SH186014,2020-07-06 16:29:28
782,0,APPLE48EDDIES,adlste_coa.coat_dat_DB_Objects,2020-06-15,2020-07-01,adlste_coa.sp_dat_DB_Objects,1.0,normal run,START,,SH186014,2020-07-06 16:29:27
781,780,APPLE62SEDONA,adlste_coa.coat_dat_DB_Objects,2020-05-22,2020-07-01,adlste_coa.sp_dat_DB_Objects,1.0,normal run,END,,SH186014,2020-07-06 16:29:22
780,0,APPLE62SEDONA,adlste_coa.coat_dat_DB_Objects,2020-05-22,2020-07-01,adlste_coa.sp_dat_DB_Objects,1.0,normal run,START,,SH186014,2020-07-06 16:29:22
779,778,APPLE57CRESCENT,adlste_coa.coat_dat_DB_Objects,2020-05-22,2020-07-01,adlste_coa.sp_dat_DB_Objects,1.0,normal run,END,,SH186014,2020-07-06 16:29:16
778,0,APPLE57CRESCENT,adlste_coa.coat_dat_DB_Objects,2020-05-22,2020-07-01,adlste_coa.sp_dat_DB_Objects,1.0,normal run,START,,SH186014,2020-07-06 16:29:16


<br/><br/><br/><br/>
# Column Type

In [ ]:
drop table adlste_coa_stg.stg_dat_column_type ;

In [ ]:
CREATE MULTISET GLOBAL TEMPORARY TABLE adlste_coa_stg.stg_dat_column_type 
( Site_ID          VARCHAR(128) CHARACTER SET LATIN
, LogDate          DATE 
, Column_Type      VARCHAR(256) CHARACTER SET LATIN 
, Column_Category  VARCHAR(128) CHARACTER SET LATIN 
, Total_Cnt        INT
)
NO PRIMARY INDEX 
ON COMMIT PRESERVE ROWS;

In [ ]:
drop table adlste_coa.coat_dat_column_type

In [ ]:
CREATE MULTISET TABLE adlste_coa.coat_dat_column_type 
( Site_ID          VARCHAR(128) CHARACTER SET LATIN
, LogDate          DATE
, Column_Type      VARCHAR(256) CHARACTER SET LATIN 
, Column_Category  VARCHAR(128) CHARACTER SET LATIN 
, Total_Cnt        INT
, Run_ID           INT
)
PRIMARY INDEX (Site_ID, LogDate);

In [ ]:
replace view adlste_coa.coa_dat_column_type
as locking row for access
select * from adlste_coa.coat_dat_column_type

In [ ]:
REPLACE PROCEDURE adlste_coa.sp_dat_column_type ( spversion VARCHAR(128) ) 
SQL SECURITY CREATOR
BEGIN 

    /* start setup audit process */
    
    DECLARE runid  INTEGER;
    DECLARE siteid VARCHAR(128);
    DECLARE tablename VARCHAR(128);
    DECLARE callingsp VARCHAR(128); 
    DECLARE startdate DATE;
    DECLARE enddate DATE;
    
    SET runid = 0;
    SET tablename = 'adlste_coa.coat_dat_column_type';
    SET callingsp = 'adlste_coa.sp_dat_column_type';
    SET siteid = 'Empty Table';
    SET startdate = DATE;
    SET enddate = DATE;

    SELECT 
     coalesce(Site_ID,'empty table')       
    ,min(LogDate) as StartDate
    ,max(LogDate) as EndDate
    INTO siteid, startdate, enddate
    FROM adlste_coa_stg.stg_dat_column_type
    Group by 1
    ;
    
    /* end setup audit process */
    
    
    
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate,
                              'normal run', 'START',  '') ;

    DELETE FROM adlste_coa.coat_dat_column_type 
    WHERE (Site_ID, LogDate) IN
    (Select Site_ID, LogDate From adlste_coa_stg.stg_dat_column_type );
    
    Insert into adlste_coa.coat_dat_column_type 
    Select
     Site_ID
    ,LogDate
    ,Column_Type
    ,Column_Category
    ,Total_Cnt
    , :runid
    from adlste_coa_stg.stg_dat_column_type as a ; 
    
    delete from adlste_coa_stg.stg_dat_column_type;
        
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate,
                              'normal run', 'END', '') ;

END;

In [ ]:
select * from adlste_coa.coat_dat_column_type

In [ ]:
select top 100 * from adlste_coa.coa_audit_log 
where TableName = 'adlste_coa.coat_dat_column_type'
order by 1 desc